# 01 - Model Client Usage

Demonstrate various usage scenarion for Model Client.

## A - Text Generation

For most of these examples, the Qwen 3 model is used since it supports both reasoning/thinking and tool usage.

In [1]:
from aimu.models import OllamaClient

model_client = OllamaClient(OllamaClient.MODELS.QWEN_3_8B)
print(model_client.model.name + "\n" + model_client.model.value)

QWEN_3_8B
qwen3:8b


For non-interactive (chat-based) generation, the *generate* method is used. This method allows you to specify a prompt and receive a generated response.

In [2]:
model_client.generate("What is the capital of France?")

'The capital of France is **Paris**. It is the political, cultural, and economic center of the country and has been the capital since the 13th century.'

Output can be streamed as well.

In [3]:
response = model_client.generate_streamed("What is the capital of England?")

for response_part in response:
    print(response_part, end="", flush=True)

The capital of England is **London**. London is also the capital of the United Kingdom, as England is one of the four countries that make up the UK.

Parameters can be passed to the generate (and chat) methods to control the output. For thinking/reasoning models, max_tokens should be large enough to capture both the reasoning output, which happens first, and the generation output.

In [4]:
# For generate_kwargs, see https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values

model_client.generate(
    "What is the capital of Germany?",
    generate_kwargs={
        "temperature": 0.1,
        "top_p": 0.9,
        "top_k": 40,
        "max_tokens": 1024,
        "repeat_penalty": 1.1,
    },
)

'The capital of Germany is **Berlin**. \n\nBerlin has been the capital since 1990 following the reunification of East and West Germany. It is also the largest city in the country and a major cultural, economic, and political hub. While other cities like Munich, Hamburg, or Frankfurt are significant for their historical or economic roles, Berlin holds the official status as the capital of Germany.'

## B - Basic Chat

Chatting with a model simply involves passing a message and receiving a response. The Model Client handles the conversation history automatically.

In [5]:
model_client.chat("Where is the Caribbean?")

"The **Caribbean** is a region located in the **western part of the Atlantic Ocean**, encompassing a vast area that includes the **Caribbean Sea**, the **Gulf of Mexico**, and parts of the **coastal regions of Central and South America**. Here's a breakdown of its geography and significance:\n\n---\n\n### **1. Geographical Location**\n- **Caribbean Sea**: The central body of water, bordered by:\n  - **North**: Mexico (Yucatán Peninsula), Cuba, and the Bahamas.\n  - **East**: The Lesser Antilles (e.g., Puerto Rico, Dominican Republic, Haiti, Jamaica).\n  - **South**: Colombia, Venezuela, and parts of Brazil.\n  - **West**: Central America (e.g., Honduras, Nicaragua, Panama).\n- **Gulf of Mexico**: To the north, connected to the Caribbean Sea via the Yucatán Channel.\n\n---\n\n### **2. Key Countries and Islands**\nThe Caribbean includes **25 countries and territories**, with notable ones being:\n- **Islands**:\n  - **Greater Antilles**: Cuba, Jamaica, Hispaniola (Haiti and the Dominican 

The complete message history, including the model thinking/resoning, can be accessed via the `messages` property of the Model Client.

In [6]:
model_client.messages

[{'role': 'user', 'content': 'Where is the Caribbean?'},
 {'role': 'assistant',
  'content': "The **Caribbean** is a region located in the **western part of the Atlantic Ocean**, encompassing a vast area that includes the **Caribbean Sea**, the **Gulf of Mexico**, and parts of the **coastal regions of Central and South America**. Here's a breakdown of its geography and significance:\n\n---\n\n### **1. Geographical Location**\n- **Caribbean Sea**: The central body of water, bordered by:\n  - **North**: Mexico (Yucatán Peninsula), Cuba, and the Bahamas.\n  - **East**: The Lesser Antilles (e.g., Puerto Rico, Dominican Republic, Haiti, Jamaica).\n  - **South**: Colombia, Venezuela, and parts of Brazil.\n  - **West**: Central America (e.g., Honduras, Nicaragua, Panama).\n- **Gulf of Mexico**: To the north, connected to the Caribbean Sea via the Yucatán Channel.\n\n---\n\n### **2. Key Countries and Islands**\nThe Caribbean includes **25 countries and territories**, with notable ones being:\n

As with the generate method, the model response can be streamed as well.

In [7]:
response = model_client.chat_streamed("Where there pirates there?")

for chunk in response:
    print(chunk, end="", flush=True)

Yes, **pirates were historically common in the Caribbean**, particularly during the **17th to 18th centuries**, when the region became a notorious hub for piracy due to its **strategic location**, **abundant resources**, and **lack of strong naval enforcement**. Here's a breakdown of where pirates were active and why the Caribbean was a pirate haven:

---

### **1. Historical Pirate Havens in the Caribbean**
- **The Bahamas**:  
  - The **Bahamas** became a major pirate base, especially for **Edward Teach** (Blackbeard) and **Bartholomew Roberts**. The **Isle of Tortuga** (now part of Haiti) and **New Providence Island** were key pirate strongholds.  
  - The **Spanish Main** (coastal regions of Central and South America) was also a target for pirates seeking to plunder Spanish treasure ships.

- **Jamaica and the West Indies**:  
  - **Jamaica** and other islands in the **West Indies** were often used as refuges for pirates. The **Cayman Islands** and **Cuba** also had pirate activity

A custom system message can be set, or re-set, as desired.

In [ ]:
model_client.system_message = "You are a helpful assistant that responds using pirate speak."
model_client.messages[0]

{'role': 'system',
 'content': 'You are a helpful assistant that responds using pirate speak.'}

In [9]:
model_client.chat("Name a famous pirate from there.")

'Arrr, ye be askin’ fer a name o’ a famous pirate from these waters? Let me tell ye ‘bout **Blackbeard**—aye, the fearsome **Edward Teach**!  \n\n**Blackbeard** was a terror o’ the seas, with his long, black beard and a face like a devil. He made his lair in the **Bahamas**, where he and his crew of scallywags would raid Spanish galleons fer their treasure. His ship, the *Queen Anne’s Revenge*, was a dreadnought o’ the Caribbean, and he’d scare the bejesus out o’ sailors with his ghostly appearance and the smoke he’d blow from his beard.  \n\nBut don’t think yer safe from the **Caymans** or the **Bahamas**—there be others too! **Bartholomew Roberts** was a swashbucklin’ scallywag who captured over 400 ships! And then there be **Anne Bonny** and **Mary Read**, two buxom lassies who fought side by side with the devil himself, ye scurvy dog!  \n\nAye, the Caribbean be a land o’ legends, and these pirates be the stuff o’ dreams and nightmares! 🏴\u200d☠️'

## C - Tool Usage

Create an MCP tool to use. See the "02 - MCP Tools" notebook for more examples of how to set up and use MCP tools.

In [10]:
import datetime
from fastmcp import FastMCP

mcp = FastMCP("AIMU Tools")


@mcp.tool()
def get_current_date_and_time() -> str:
    """Returns the current data and time as a string."""
    return datetime.datetime.now()

Create a new AIMU MCPClient for handling MCP tool requests.

In [11]:
from aimu.tools import MCPClient

# Required to allow nested event loops in Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

mcp_client = MCPClient(mcp)
model_client.mcp_client = mcp_client

In [14]:
model_client.chat("What time is it?", tools=mcp_client.get_tools())

ValidationError: 1 validation error for Message
content
  Input should be a valid string [type=string_type, input_value={'role': 'user', 'content': 'What time is it?'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [ ]:
model_client.messages

In [ ]:
response = model_client.chat_streamed("What time is it now?", tools=mcp_client.get_tools())

for chunk in response:
    print(chunk, end="", flush=True)

In [ ]:
model_client.messages